# Tengo q hacer esto pq no me está andando el script original. En algún momento lo arreglaré

In [1]:
#! /usr/local/env julia
###############################################################################
# Utility to displace a PDB along many vectors, generating 1 PDB for each
# vector, useful for NDD analysis.
# Calpha only.
# by https://github.com/pgbarletta
###############################################################################
using Chemfiles, JUMD
using StaticArrays
using ArgParse
using DelimitedFiles, LinearAlgebra, FileIO

function displaceAA(in_frm, aa, aa_3, in_vec)
    # Preparo variables
    in_top = Topology(in_frm)
    natoms = convert(Int64, size(in_top))
    in_xyz = positions(in_frm)

    # Determino orden de residuos (hay q actualizar el Julia Chemfiles)
    tmp = Array{Int64}(undef, aa)
    ids = Array{Int64}(undef, aa)
    [ ids[i+1] = convert(Int64, id((Residue(in_top, i)))) for i = 0:aa-1 ]
    idx = sortperm(ids)
    # Determino el nro de atomos de c/ aminoácido
    [ tmp[i+1] = size(Residue(in_top, i)) for i = 0:aa-1 ]
    natom_aa = tmp[idx]

    # Paso el vector columna de tamaño 1xaa_3 a 3xaa
    vector = reshape(in_vec, 3, aa)
    # Adapto el vector p/ darle la misma forma q la matriz de coordenadas
    sum_mat = Array{Float64}(undef, 3, natoms)
    cursor = 0
    for i = 1:aa
        if i == 1
            sum_mat[:, 1:natom_aa[i]] = repeat(vector[:, 1], 1, natom_aa[i])
            cursor = natom_aa[i]
            continue
        end
        rango = collect(cursor+1:cursor + natom_aa[i])
        sum_mat[:, rango] = repeat(vector[:, i], 1, natom_aa[i])
        cursor += natom_aa[i]
    end

    # Listo, ahora puedo mover el pdb
    out_frm = deepcopy(in_frm)
    out_xyz = positions(out_frm)

    # Tengo q hacer esto por ahora. Hasta q arreglemos Chemfiles.
    for i = 1:size(in_xyz)[1]
        for j = 1:size(in_xyz)[2]
            out_xyz[i, j]  = round(in_xyz[i, j] + sum_mat[i, j], digits = 3)
        end
    end
    return out_frm
end

displaceAA (generic function with 1 method)

In [6]:
in_pdb_filename = "../avg_1ifb.pdb"
modes_filename = "../../../pca/3/3vol_modes_1ifb"
mul = 5
suffix = "1ifb"
amber_modes = true
weights_filename = "none"

# Read PDB
const in_trj = Trajectory(in_pdb_filename)
const in_frm = read(in_trj)
const in_top = Topology(in_frm)
const aa = convert(Int64, count_residues(in_top))
const aa_3 = aa * 3
nmodes = aa_3 - 6

354

In [7]:
# Read input vectors and weights
in_modes, weights = JUMD.readPtrajModes(modes_filename);

In [8]:
# Ahora desplazo
pdb_names = Array{String}(undef, nmodes)
for i = 1:nmodes
    # Escalo vector
    modo = in_modes[:, i] .* mul ./ weights[i]
    # Desplazo
    out_frm = displaceAA(in_frm, aa, aa_3, modo);
    # Y guardo
    pdb_names[i] = joinpath(pwd(), string(i, "_", suffix, ".pdb"))
    out_trj = Trajectory(pdb_names[i], 'w')
    write(out_trj, out_frm)
    close(out_trj)
end

In [9]:
# Write in_ndd file
in_ndd_filename = string("in_ndd_", splitext(basename(in_pdb_filename))[1])
in_ndd_location = dirname(in_pdb_filename)
writedlm(joinpath(in_ndd_location, in_ndd_filename), pdb_names)